In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('../util')
sys.path.append('../experiments')

In [4]:
from convert_annotations import read_webanno, resolve_ellipses, webanno_to_iob_df, EMPTY_REGEX
from pathlib import Path
from datetime import datetime
import pandas as pd

In [5]:
%%time
tsv_folder = '../../data_ggponc/output_ellipses/'
#tsv_folder = '../../ggponc_v2_annotation/other_layers/ellipses_dev'

webanno_df, sentences = read_webanno(list(Path(tsv_folder).glob('*.tsv')))

CPU times: user 12.6 s, sys: 682 ms, total: 13.3 s
Wall time: 20.5 s


In [6]:
%%time

import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    iob_df = webanno_to_iob_df(webanno_df, 'detail', True, debug=False, collect_errors=False, skip_errors=True, all_columns=True)
    iob_df['id'] = iob_df.entity_id.where(~iob_df.entity_id.isna(), iob_df.spec_id)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 49821/49821 [18:24<00:00, 45.11it/s]


CPU times: user 18min 34s, sys: 9.91 s, total: 18min 44s
Wall time: 18min 42s


In [10]:
# Anything containing a prefix, suffix or fragment is potentially an elliptical construct
ellipses = iob_df[~iob_df.prefix.str.match(EMPTY_REGEX) | ~iob_df.suffix.str.match(EMPTY_REGEX) | ~iob_df.fragment.str.match(EMPTY_REGEX)].copy()

In [11]:
iob_df

token_id ts_id     span  \
file                                  sentence_id                            
00_mundhoehlenkarzinom_0002.tsv       1                   1   1-1     0-19   
                                      1                   2   1-2    20-23   
                                      1                   3   1-3    24-27   
                                      1                   4   1-4    28-37   
                                      1                   5   1-5    38-43   
...                                                     ...   ...      ...   
29_adulte-weichgewebesarkome_0405.tsv 4                  26  4-26  755-756   
                                      4                  27  4-27  757-767   
                                      4                  28  4-28  768-779   
                                      4                  29  4-29  779-780   
                                      4                  30  4-30  780-781   

                                                                 token  \
file                                  sentence_id                        
00_mundhoehlenkarzinom_0002.tsv       1            Hauptrisikofaktoren   
                                      1                            für   
                                      1                            das   
                                      1                      Auftreten   
                                      1                          eines   
...                                                                ...   
29_adulte-weichgewebesarkome_0405.tsv 4                              .   
                                      4                     lipotrophe   
                                      4                    Chemikalien   
                                      4                              )   
                                      4                              .   

                                                              value  \
file                                  sentence_id                     
00_mundhoehlenkarzinom_0002.tsv       1                     Finding   
                                      1                           _   
                                      1                           _   
                                      1                     Finding   
                                      1            Specification[1]   
...                                                             ...   
29_adulte-weichgewebesarkome_0405.tsv 4                           _   
                                      4               Specification   
                                      4                   Substance   
                                      4                           _   
                                      4                           _   

                                                                   detail  \
file                                  sentence_id                           
00_mundhoehlenkarzinom_0002.tsv       1                     Other Finding   
                                      1                                 _   
                                      1                                 _   
                                      1            Diagnosis or Pathology   
                                      1                              *[1]   
...                                                                   ...   
29_adulte-weichgewebesarkome_0405.tsv 4                                 _   
                                      4                                 *   
                                      4                External Substance   
                                      4                                 _   
                                      4                                 _   

                                                  specified_by prefix suffix  \
file                                  sentence_id                  

In [12]:
%%time
results = resolve_ellipses(ellipses, iob_df)

../util/convert_annotations.py:166: PerformanceWarning: indexing past lexsort depth may impact performance.
  sentence = ellipses.loc[idx]
../util/convert_annotations.py:168: PerformanceWarning: indexing past lexsort depth may impact performance.
  full_sentence = iob_df.loc[idx]


CPU times: user 2min 11s, sys: 1.31 s, total: 2min 12s
Wall time: 2min 12s


# Manual fixes

In [461]:
#idx = results[(results.file == '01_magenkarzinom_0090.tsv') & (results.sentence_id == 22)].index
#assert len(idx) == 1
#results.at[idx[0], 'resolution'] = ['Stadium', '2', 'und', 'Stadium 3']

In [479]:
sentence_idx = pd.DataFrame(sentences, columns=['raw_sentence'], index=webanno_df.index.drop_duplicates())

In [480]:
clean_results = results.merge(right=sentence_idx, left_on=['file', 'sentence_id'], right_index=True)

In [481]:
for _, k in clean_results[clean_results.duplicated(['file', 'sentence_id'])][['file', 'sentence_id']].iterrows():
    f = k['file']
    s = k['sentence_id']
    dups = clean_results[(clean_results.file == f) & (clean_results.sentence_id == s)]
    
    for i, inner in dups.iterrows():
        for j, outer in dups.iterrows():
            if i != j and inner.span_index_start >= outer.span_index_start and inner.span_index_end <= outer.span_index_end:
                clean_results.loc[i, 'outer'] = j

In [482]:
clean_results = clean_results[clean_results.outer.isna()]

In [483]:
def resolve(row):
    out = {}
    for c in row.index:
        val = row.loc[c]
        if len(val) == 1:
            out['multi'] = False
            out[c] = val[0]
        else:
            out['multi'] = True
            if val.count(val[0]) == len(val) and c in ['full_sentence', 'raw_sentence']:
                out[c] = val[0]
            elif c in ['fragment', 'missing_prefix', 'missing_suffix']:
                out[c] = max(val)
            else:
                out[c] = val
    return pd.Series(out)

In [484]:
clean_results = clean_results.groupby(['file', 'sentence_id']).aggregate(tuple).applymap(list).apply(resolve, axis=1)[list(clean_results.columns.drop(['outer', 'file', 'sentence_id'])) + ['multi']].reset_index()

In [485]:
def get_full_res(row):
    sent = row.raw_sentence
    if row.multi:
        spans = [o for p in row.offsets for o in p]
        resolution = [r for p in row.resolution for r in p]
    else:
        spans = row.offsets
        resolution = row.resolution
    assert len(spans) == len(resolution), (spans, resolution)
    j = 0
    for span, res in zip(spans, resolution):
        start, end = span
        start += j
        end += j
        sent = sent[:start] + res + sent[end:]
        j += (start - end + len(res))
    return sent

In [486]:
clean_results['full_resolution'] = clean_results.apply(get_full_res, axis=1)

In [487]:
import numpy as np

In [488]:
ellipses_sentence_count = len(results[['file', 'sentence_id']].drop_duplicates())
sentence_count = 78090
ellipses_sentence_count, sentence_count, ellipses_sentence_count / sentence_count

(4622, 78090, 0.05918811627609169)

In [489]:
print('Ellipses', len(results))
print('Forward ellipses', len(results[results.missing_prefix & ~results.missing_suffix]), len(results[results.fragment & results.missing_prefix & ~results.missing_suffix]))
print('Backward ellipses', len(results[~results.missing_prefix & results.missing_suffix]), len(results[results.fragment & ~results.missing_prefix & results.missing_suffix]))
print('Complex ellipses', len(results[results.missing_prefix & results.missing_suffix]), len(results[results.fragment & results.missing_prefix & results.missing_suffix]))

Ellipses 5196
Forward ellipses 1933 833
Backward ellipses 2961 2751
Complex ellipses 266 185


In [501]:
import random

def show_random(df):
    i = random.randint(0, len(df) - 1)
    item = df.iloc[i]    
    assert len(item.full_span) == len(item.resolution)
    spans = []
    solutions = []
    marker = []
    if not item.multi:
        for span, res in zip(item.full_span, item.resolution):
            if span == res:
                marker.append(' ' * len(span))
            else:
                marker.append('^' * len(res))
            start_index = res.find(span)
            spans.append(span.rjust(start_index + len(span)).ljust(len(res) - start_index))
            solutions.append(res)
        print(' '.join(spans))
        print(' '.join(solutions))
        print(' '.join(marker))
        print(f'{item.file};{item.sentence_id};{" ".join(item.full_span)}', 'fragment:', item.fragment)
    print(item.raw_sentence)
    print(item.full_resolution)
    print('...........')
    print('')

In [491]:
for _ in range(0, 10):
    show_random(clean_results[clean_results.multi])

- N (pN0 und pNx vs. pN1-2)
- N-Kategorie (pN0 und pNx vs. pN1pN2)
...........

Nach vorausgegangener Radio- und/oder Chemotherapie stellt das Ausmaß der therapieinduzierten Tumorregression im Bereich des Primärtumors und der regionären Lymphknoten einen unabhängigen Prognosefaktor dar [REF], [REF].
Nach vorausgegangener Radiotherapie und/oder Chemotherapie stellt das Ausmaß der therapieinduzierten Tumorregression im Bereich des Primärtumors und im Bereich der regionären Lymphknoten einen unabhängigen Prognosefaktor dar [REF], [REF].
...........

373 Krebspatienten mit verschiedenen Karzinomen erhielten zusätzlich zur Chemo- oder Radiotherapie entweder eine individuelle homöopathische Behandlung mit Globuli und Dilutionen oder nur Chemo- oder Radiotherapie.
373 Krebspatienten mit verschiedenen Karzinomen erhielten zusätzlich zur Chemotherapie oder Radiotherapie entweder eine individuelle homöopathische Behandlung mit Globuli und Dilutionen oder nur Chemotherapie oder Radiotherapie.
...

In [492]:
for _ in range(0, 10):
    show_random(clean_results[clean_results.fragment])

relevante Darm                 - oder Lungenvorerkrankungen
relevante Darm-vorerkrankungen  oder Lungenvorerkrankungen
          ^^^^^^^^^^^^^^^^^^^^                            
19_follikulaeres-lymphom_0237.tsv;10;relevante Darm - oder Lungenvorerkrankungen fragment: True
Da gerade in der Gruppe der doppelt-refraktären Patienten das 1-Jahres PFS ebenfalls bei 66% liegt, stellt Lenalidomid und Rituximab eine nicht zugelassene Therapieoption für Patienten dar, bei denen eine Idelalisibtherapie nicht durchführbar ist (z.B. relevante Darm- oder Lungenvorerkrankungen).
Da gerade in der Gruppe der doppelt-refraktären Patienten das 1-Jahres PFS ebenfalls bei 66% liegt, stellt Lenalidomid und Rituximab eine nicht zugelassene Therapieoption für Patienten dar, bei denen eine Idelalisibtherapie nicht durchführbar ist (z.B. relevante Darm-vorerkrankungen oder Lungenvorerkrankungen).
...........

Muskelerhalt und -       aufbau
Muskelerhalt und  Muskelaufbau
                  ^^^^^^^^^^^^
01_magen

In [493]:
ss = clean_results[~clean_results.fragment][clean_results[~clean_results.fragment].resolution.map(lambda r: any([t for t in r if '-' in t]))]

In [494]:
for _ in range(0, 10):
    show_random(ss)

endoskopisch              / lokal abgetragenen low-risk T1 - Karzinomen
endoskopisch abgetragenen / lokal abgetragenen low-risk T1 - Karzinomen
^^^^^^^^^^^^^^^^^^^^^^^^^                                              
02_kolorektales-karzinom_0079.tsv;26;endoskopisch / lokal abgetragenen low-risk T1 - Karzinomen fragment: False
Obligate Voraussetzung für eine Nachbeobachtung ohne onkologische Resektion bei endoskopisch/lokal abgetragenen low-risk T1-Karzinomen ist die sichere Beurteilung der vollständigen Abtragung (R0).
Obligate Voraussetzung für eine Nachbeobachtung ohne onkologische Resektion bei endoskopisch abgetragenen/lokal abgetragenen low-risk T1-Karzinomen ist die sichere Beurteilung der vollständigen Abtragung (R0).
...........

Frauen mit unauffälliger Zytologie und     persistierendem HPV-Nachweis
Frauen mit unauffälliger Zytologie und mit persistierendem HPV-Nachweis
                                       ^^^^^^^^^^^^^^^^^^^             
04_praevention-zervixkarzinom_0106.t

In [495]:
for _ in range(0, 10):
    show_random(clean_results[~clean_results.fragment])

Beratung durch eine spezialisierte Pflegekraft oder       psychosoziale Fachkräfte
Beratung durch eine spezialisierte Pflegekraft oder durch psychosoziale Fachkräfte
                                                    ^^^^^^^^^^^^^^^^^^^           
07_malignes-melanom_0069.tsv;15;Beratung durch eine spezialisierte Pflegekraft oder psychosoziale Fachkräfte fragment: False
Angebote weitergehender Beratung durch eine spezialisierte Pflegekraft oder psychosoziale Fachkräfte tragen zu besserem Verständnis und Behalten der erhaltenen Informationen bei.
Angebote weitergehender Beratung durch eine spezialisierte Pflegekraft oder durch psychosoziale Fachkräfte tragen zu besserem Verständnis und Behalten der erhaltenen Informationen bei.
...........

Progrediente Tumoren unter Sulindac oder       antihormoneller Therapie
Progrediente Tumoren unter Sulindac oder unter antihormoneller Therapie
                                         ^^^^^^^^^^^^^^^^^^^^^         
02_kolorektales-karzinom_0162.tsv

In [504]:
date = datetime.now().strftime("%Y%m%d_%H%M%S")

In [505]:
date

'20220803_195531'

In [506]:
clean_results.to_excel(f'ellipses_no_dups_{date}.xlsx')